# 第3章：DataFrame 和 Dataset API

本章節將深入學習 DataFrame 和 Dataset 的操作，這是 Spark 中最常用的資料結構。

## 學習目標
- 理解 DataFrame 的概念和優勢
- 掌握 DataFrame 的基本操作
- 學習 Dataset API 的使用
- 了解 Catalyst 優化器的工作原理

## 1. 環境設置

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, upper, lower, substring, concat
from pyspark.sql.functions import sum as spark_sum, avg, max as spark_max, min as spark_min, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from datetime import datetime, date

In [ ]:
# 創建 SparkSession
spark = SparkSession.builder \
    .appName("DataFrame操作學習") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"Spark版本: {spark.version}")
print(f"Spark UI: http://localhost:4040")

## 2. 創建 DataFrame

In [ ]:
# 方法1：從 Python 數據創建
employees_data = [
    ("Alice", 25, "Engineering", 75000, datetime(2020, 1, 15)),
    ("Bob", 30, "Sales", 65000, datetime(2019, 3, 20)),
    ("Charlie", 35, "Engineering", 85000, datetime(2018, 7, 10)),
    ("Diana", 28, "Marketing", 60000, datetime(2021, 2, 5)),
    ("Eve", 32, "Engineering", 80000, datetime(2019, 11, 12)),
    ("Frank", 29, "Sales", 70000, datetime(2020, 8, 30)),
    ("Grace", 26, "Marketing", 55000, datetime(2021, 5, 18)),
    ("Henry", 31, "Engineering", 90000, datetime(2017, 4, 22))
]

columns = ["name", "age", "department", "salary", "hire_date"]
employees_df = spark.createDataFrame(employees_data, columns)

print("員工資料:")
employees_df.show()

In [ ]:
# 方法2：使用明確的 Schema
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("department", StringType(), True),
    StructField("salary", DoubleType(), True),
    StructField("hire_date", TimestampType(), True)
])

employees_df_schema = spark.createDataFrame(employees_data, schema)

print("DataFrame Schema:")
employees_df_schema.printSchema()

## 3. 基本 DataFrame 操作

In [ ]:
# 選擇特定列
print("選擇姓名和薪水:")
employees_df.select("name", "salary").show()

print("\n使用 col() 函數:")
employees_df.select(col("name"), col("salary")).show(5)

In [ ]:
# 過濾操作
print("薪水大於70000的員工:")
high_salary_df = employees_df.filter(col("salary") > 70000)
high_salary_df.show()

print("\n工程部門的員工:")
engineering_df = employees_df.filter(col("department") == "Engineering")
engineering_df.show()

In [ ]:
# 複合條件過濾
print("工程部門且薪水大於75000的員工:")
complex_filter_df = employees_df.filter(
    (col("department") == "Engineering") & (col("salary") > 75000)
)
complex_filter_df.show()

print("\n年齡在25-30歲之間的員工:")
age_range_df = employees_df.filter(
    (col("age") >= 25) & (col("age") <= 30)
)
age_range_df.show()

## 4. 添加和修改列

In [ ]:
# 添加新列
employees_with_bonus = employees_df.withColumn(
    "bonus", col("salary") * 0.1
)

print("添加獎金列:")
employees_with_bonus.select("name", "salary", "bonus").show()

# 添加年薪列
employees_with_annual = employees_with_bonus.withColumn(
    "annual_salary", col("salary") * 12
)

print("\n添加年薪列:")
employees_with_annual.select("name", "salary", "annual_salary").show()

In [ ]:
# 條件列
employees_with_level = employees_df.withColumn(
    "level",
    when(col("salary") >= 80000, "Senior")
    .when(col("salary") >= 65000, "Mid")
    .otherwise("Junior")
)

print("員工等級:")
employees_with_level.select("name", "salary", "level").show()

# 年齡組分類
employees_with_age_group = employees_with_level.withColumn(
    "age_group",
    when(col("age") < 28, "Young")
    .when(col("age") < 32, "Middle")
    .otherwise("Senior")
)

print("\n年齡組分類:")
employees_with_age_group.select("name", "age", "age_group", "level").show()

## 5. 分組和聚合操作

In [ ]:
# 按部門分組統計
dept_stats = employees_df.groupBy("department").agg(
    count("*").alias("employee_count"),
    avg("salary").alias("avg_salary"),
    spark_max("salary").alias("max_salary"),
    spark_min("salary").alias("min_salary"),
    spark_sum("salary").alias("total_salary")
)

print("部門統計:")
dept_stats.show()

# 按年齡組分組
age_group_stats = employees_with_age_group.groupBy("age_group").agg(
    count("*").alias("count"),
    avg("salary").alias("avg_salary")
)

print("\n年齡組統計:")
age_group_stats.show()

In [ ]:
# 多維度分組
dept_level_stats = employees_with_level.groupBy("department", "level").agg(
    count("*").alias("count"),
    avg("salary").alias("avg_salary")
)

print("部門和等級統計:")
dept_level_stats.orderBy("department", "level").show()

## 6. 排序操作

In [ ]:
# 按薪水排序
print("按薪水升序排序:")
employees_df.orderBy("salary").show()

print("\n按薪水降序排序:")
employees_df.orderBy(col("salary").desc()).show()

# 多列排序
print("\n按部門和薪水排序:")
employees_df.orderBy("department", col("salary").desc()).show()

## 7. 字符串操作

In [ ]:
# 字符串轉換
string_ops_df = employees_df.select(
    "name",
    upper(col("name")).alias("name_upper"),
    lower(col("name")).alias("name_lower"),
    substring(col("name"), 1, 3).alias("name_substr"),
    concat(col("name"), lit(" - "), col("department")).alias("name_dept")
)

print("字符串操作:")
string_ops_df.show()

## 8. 連接操作

In [ ]:
# 創建部門資料
dept_data = [
    ("Engineering", "Tech", "Building A"),
    ("Sales", "Business", "Building B"),
    ("Marketing", "Business", "Building C"),
    ("HR", "Support", "Building D")
]

dept_df = spark.createDataFrame(dept_data, ["department", "division", "location"])

print("部門資料:")
dept_df.show()

# Inner Join
print("\n員工和部門資料連接:")
joined_df = employees_df.join(dept_df, "department")
joined_df.select("name", "department", "division", "location", "salary").show()

# Left Join
print("\n左連接 (保留所有員工):")
left_joined_df = employees_df.join(dept_df, "department", "left")
left_joined_df.select("name", "department", "division", "location").show()

## 9. 視窗函數

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, lag, lead

# 定義視窗規格
window_spec = Window.partitionBy("department").orderBy(col("salary").desc())

# 添加排名
ranked_df = employees_df.withColumn(
    "rank", rank().over(window_spec)
).withColumn(
    "dense_rank", dense_rank().over(window_spec)
).withColumn(
    "row_number", row_number().over(window_spec)
)

print("部門內薪水排名:")
ranked_df.select("name", "department", "salary", "rank", "dense_rank", "row_number").show()

# 找出每個部門薪水最高的員工
top_earners = ranked_df.filter(col("rank") == 1)
print("\n每個部門薪水最高的員工:")
top_earners.select("name", "department", "salary").show()

## 10. 缺失值處理

In [ ]:
# 創建包含缺失值的數據
data_with_nulls = [
    ("Alice", 25, "Engineering", 75000),
    ("Bob", None, "Sales", 65000),
    ("Charlie", 35, None, 85000),
    ("Diana", 28, "Marketing", None),
    (None, 32, "Engineering", 80000)
]

df_with_nulls = spark.createDataFrame(data_with_nulls, ["name", "age", "department", "salary"])

print("包含缺失值的數據:")
df_with_nulls.show()

# 檢查缺失值
print("\n各列缺失值統計:")
for col_name in df_with_nulls.columns:
    null_count = df_with_nulls.filter(col(col_name).isNull()).count()
    total_count = df_with_nulls.count()
    print(f"{col_name}: {null_count} / {total_count} ({null_count/total_count*100:.1f}%)")

In [ ]:
# 刪除包含缺失值的行
print("刪除包含缺失值的行:")
df_dropped = df_with_nulls.dropna()
df_dropped.show()

# 填充缺失值
print("\n填充缺失值:")
df_filled = df_with_nulls.fillna({
    "name": "Unknown",
    "age": 30,
    "department": "Unknown",
    "salary": 50000
})
df_filled.show()

## 11. 資料類型轉換

In [ ]:
# 檢查當前資料類型
print("當前資料類型:")
employees_df.printSchema()

# 轉換資料類型
converted_df = employees_df.withColumn(
    "salary_str", col("salary").cast("string")
).withColumn(
    "age_double", col("age").cast("double")
)

print("\n轉換後的資料類型:")
converted_df.printSchema()

# 顯示轉換結果
converted_df.select("name", "salary", "salary_str", "age", "age_double").show(5)

## 12. 執行計劃分析

In [ ]:
# 查看執行計劃
complex_query = employees_df.filter(col("salary") > 70000) \
                           .groupBy("department") \
                           .agg(avg("salary").alias("avg_salary")) \
                           .orderBy(col("avg_salary").desc())

print("執行計劃:")
complex_query.explain()

print("\n詳細執行計劃:")
complex_query.explain(True)

## 13. 緩存和持久化

In [ ]:
# 緩存 DataFrame
cached_df = employees_df.cache()

# 觸發緩存
print("觸發緩存操作:")
print(f"總員工數: {cached_df.count()}")
print(f"平均薪水: {cached_df.agg(avg('salary')).collect()[0][0]:.2f}")

# 檢查緩存狀態
print(f"\n是否已緩存: {cached_df.is_cached}")

# 取消緩存
cached_df.unpersist()
print(f"取消緩存後: {cached_df.is_cached}")

## 14. 實戰練習

### 練習1：員工薪水分析
分析員工薪水分佈，找出薪水異常值

In [ ]:
# 計算薪水統計
salary_stats = employees_df.agg(
    avg("salary").alias("avg_salary"),
    spark_max("salary").alias("max_salary"),
    spark_min("salary").alias("min_salary")
).collect()[0]

print("薪水統計:")
print(f"平均薪水: ${salary_stats['avg_salary']:,.2f}")
print(f"最高薪水: ${salary_stats['max_salary']:,.2f}")
print(f"最低薪水: ${salary_stats['min_salary']:,.2f}")

# 找出薪水異常值（高於平均薪水 1.5 倍的員工）
avg_salary = salary_stats['avg_salary']
outliers = employees_df.filter(col("salary") > avg_salary * 1.5)

print("\n薪水異常值員工:")
outliers.select("name", "department", "salary").show()

### 練習2：部門薪水差異分析
分析不同部門之間的薪水差異

In [ ]:
# 計算部門薪水統計
dept_salary_analysis = employees_df.groupBy("department").agg(
    count("*").alias("employee_count"),
    avg("salary").alias("avg_salary"),
    spark_max("salary").alias("max_salary"),
    spark_min("salary").alias("min_salary"),
    (spark_max("salary") - spark_min("salary")).alias("salary_range")
)

print("部門薪水分析:")
dept_salary_analysis.orderBy(col("avg_salary").desc()).show()

# 找出薪水範圍最大的部門
max_range_dept = dept_salary_analysis.orderBy(col("salary_range").desc()).first()
print(f"\n薪水範圍最大的部門: {max_range_dept['department']} (${max_range_dept['salary_range']:,.2f})")

### 練習3：年齡與薪水關係分析
分析年齡與薪水之間的關係

In [ ]:
# 創建年齡組
age_salary_analysis = employees_df.withColumn(
    "age_group",
    when(col("age") < 28, "20-27")
    .when(col("age") < 32, "28-31")
    .otherwise("32+")
).groupBy("age_group").agg(
    count("*").alias("count"),
    avg("salary").alias("avg_salary"),
    avg("age").alias("avg_age")
)

print("年齡組薪水分析:")
age_salary_analysis.orderBy("avg_age").show()

# 計算年齡與薪水的相關性（簡化版）
age_salary_corr = employees_df.select("age", "salary").rdd.map(
    lambda row: (row.age, row.salary)
).collect()

ages = [x[0] for x in age_salary_corr]
salaries = [x[1] for x in age_salary_corr]

# 簡單的相關性計算
import numpy as np
correlation = np.corrcoef(ages, salaries)[0,1]
print(f"\n年齡與薪水的相關係數: {correlation:.3f}")

## 15. 總結和清理

In [ ]:
# 顯示最終統計
print("=== DataFrame 操作學習總結 ===")
print(f"總員工數: {employees_df.count()}")
print(f"部門數: {employees_df.select('department').distinct().count()}")
print(f"薪水範圍: ${employees_df.agg(spark_min('salary')).collect()[0][0]:,} - ${employees_df.agg(spark_max('salary')).collect()[0][0]:,}")
print(f"平均薪水: ${employees_df.agg(avg('salary')).collect()[0][0]:,.2f}")

# 停止 SparkSession
spark.stop()
print("\nSpark session 已停止")

## 學習重點回顧

1. **DataFrame 基本操作**：select, filter, withColumn, groupBy, orderBy
2. **聚合函數**：count, sum, avg, max, min
3. **字符串操作**：upper, lower, substring, concat
4. **條件操作**：when, otherwise
5. **連接操作**：inner join, left join, right join
6. **視窗函數**：rank, dense_rank, row_number
7. **缺失值處理**：dropna, fillna
8. **資料類型轉換**：cast
9. **性能優化**：cache, explain
10. **實戰分析**：薪水分析、部門比較、相關性分析

通過這些練習，您已經掌握了 DataFrame 的核心操作，可以進行各種數據分析任務。